[Pyspark blog](https://bryancutler.github.io/)

[ML w/ Pyspark](https://towardsdatascience.com/machine-learning-with-pyspark-and-mllib-solving-a-binary-classification-problem-96396065d2aa)

[GitHub event types](https://developer.github.com/v3/activity/events/types/)

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, datediff

In [2]:
#spark1 = SparkSession.builder.getOrCreate()

spark = SparkSession.builder.appName('gh-churn').getOrCreate()
first_period = spark.read.csv('events_data/events_2016_01_01_2016_06_01.csv', header = True, inferSchema = True)
first_period.printSchema()

root
 |-- actor: string (nullable = true)
 |-- event_count: integer (nullable = true)
 |-- last_event: string (nullable = true)
 |-- first_event: string (nullable = true)
 |-- CheckRunEvent_count: integer (nullable = true)
 |-- CheckSuiteEvent_count: integer (nullable = true)
 |-- CommitCommentEvent_count: integer (nullable = true)
 |-- ContentReferenceEvent_count: integer (nullable = true)
 |-- CreateEvent_count: integer (nullable = true)
 |-- DeleteEvent_count: integer (nullable = true)
 |-- DeploymentEvent_count: integer (nullable = true)
 |-- DeploymentStatusEvent_count: integer (nullable = true)
 |-- DownloadEvent_count: integer (nullable = true)
 |-- FollowEvent_count: integer (nullable = true)
 |-- ForkEvent_count: integer (nullable = true)
 |-- ForkApplyEvent_count: integer (nullable = true)
 |-- asGitHubAppAuthorizationEvent_count: integer (nullable = true)
 |-- GistEvent_count: integer (nullable = true)
 |-- GollumEvent_count: integer (nullable = true)
 |-- InstallationEvent_

In [3]:
pd.DataFrame(first_period.take(5), columns=first_period.columns)

,actor,event_count,last_event,first_event,CheckRunEvent_count,CheckSuiteEvent_count,CommitCommentEvent_count,ContentReferenceEvent_count,CreateEvent_count,DeleteEvent_count,...,PushEvent_count,ReleaseEvent_count,RepositoryEvent_count,RepositoryImportEvent_count,RepositoryVulnerabilityAlertEvent_count,SecurityAdvisoryEvent_count,StatusEvent_count,TeamEvent_count,TeamAddEvent_count,WatchEvent_count
0,luceiro,1,2016-02-29 16:42:12 UTC,2016-02-29 16:42:12 UTC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,sweilemj,1,2016-02-24 21:37:39 UTC,2016-02-24 21:37:39 UTC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,JimHallett,1,2016-03-03 05:03:29 UTC,2016-03-03 05:03:29 UTC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SecurNET,1,2016-03-03 22:20:06 UTC,2016-03-03 22:20:06 UTC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,mruthvend2l,1,2016-02-23 21:12:11 UTC,2016-02-23 21:12:11 UTC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
first_period.count()

3699852

In [5]:
second_period = spark.read.csv('events_data/events_2016_01_01_2016_06_01.csv', header = True, inferSchema = True)

In [19]:
users = spark.read.csv('all_users.csv', header=True, inferSchema=True)
users = users.drop('event_count').drop('last_event').drop('first_event').drop('_c0')
pd.DataFrame(users.take(5), columns=users.columns)

,_c0,login,followers_count,following_count,bio,blog,company,created_at,public_repos_count,public_gists_count,hireable,updated_at,time_between_first_last_event
0,06Birch,06Birch,0,0,None,None,None,2016-02-08 11:19:54+00:00,0,0,None,2016-02-08 11:19:54+00:00,0 days 00:19:06.000000000
1,06peng,06peng,78,27,None,http://06peng.com,None,2013-07-25 13:43:38+00:00,1,0,True,2019-01-08 01:55:29+00:00,147 days 07:31:20.000000000
2,0p3ns0urc3,0p3ns0urc3,0,0,None,None,None,2016-04-13 22:30:01+00:00,2,0,None,2016-04-13 22:30:01+00:00,2 days 17:17:36.000000000
3,100341976,100341976,0,0,None,None,None,2016-06-01 19:30:21+00:00,0,0,None,2016-06-01 20:31:02+00:00,0 days 01:16:16.000000000
4,100amps,100amps,0,0,None,None,None,2016-03-24 05:53:08+00:00,0,0,None,2016-05-28 03:05:32+00:00,0 days 00:16:47.000000000


In [20]:
churn_data = users.join(first_period, users['login'] == first_period['actor'], how='left')
pd.DataFrame(churn_data.take(5), columns=churn_data.columns)

,_c0,login,followers_count,following_count,bio,blog,company,created_at,public_repos_count,public_gists_count,...,PushEvent_count,ReleaseEvent_count,RepositoryEvent_count,RepositoryImportEvent_count,RepositoryVulnerabilityAlertEvent_count,SecurityAdvisoryEvent_count,StatusEvent_count,TeamEvent_count,TeamAddEvent_count,WatchEvent_count
0,caussatjerome,caussatjerome,0,1,Passionné d'informatique et du développement d...,https://www.picorfi.fr,@PicOrdi,2016-04-21 13:27:17+00:00,2,0,...,0,0,0,0,0,0,0,0,0,0
1,100341976,100341976,0,0,None,None,None,2016-06-01 19:30:21+00:00,0,0,...,0,0,0,0,0,0,0,0,0,0
2,06peng,06peng,78,27,None,http://06peng.com,None,2013-07-25 13:43:38+00:00,1,0,...,0,0,0,0,0,0,0,0,0,42
3,mckenzie1993,mckenzie1993,0,0,None,None,None,2016-03-01 08:41:58+00:00,1,0,...,15,0,0,0,0,0,0,0,0,1
4,yuxian1996,yuxian1996,0,0,None,None,None,2014-07-05 01:52:12+00:00,8,20,...,0,0,0,0,0,0,0,0,0,0


In [21]:
second_period_event_count = second_period.selectExpr("actor as login",
                                                     "event_count as second_period_event_count")
second_period_event_count.show()

+------------------+-------------------------+
|             login|second_period_event_count|
+------------------+-------------------------+
|           luceiro|                        1|
|          sweilemj|                        1|
|        JimHallett|                        1|
|          SecurNET|                        1|
|       mruthvend2l|                        1|
|            sigcuk|                        1|
|      yuzdeyuzmert|                        1|
|        Kerrys7777|                        1|
|         abman2014|                        1|
|           mfwhite|                        1|
|       nguyenkynam|                        1|
|             rdjah|                        1|
|       Trovador237|                        1|
|      fernthitinan|                        1|
|          Panoktok|                        1|
|      blakestorm11|                        1|
|            srcbld|                        1|
|        khavariali|                        1|
|SonadoraComp

In [22]:
churn_data = churn_data.join(second_period_event_count, on='login', how='left')

In [23]:
numeric_features = [t[0] for t in churn_data.dtypes if t[1] == 'int']
churn_data.select(numeric_features).describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
event_count,94149,40.02193331846329,505.90596912410456,2,146454
CheckRunEvent_count,94149,0.0,0.0,0,0
CheckSuiteEvent_count,94149,0.0,0.0,0,0
CommitCommentEvent_count,94149,0.16521683714112737,3.2239640400540583,0,603
ContentReferenceEvent_count,94149,0.0,0.0,0,0
CreateEvent_count,94149,5.22832956271442,30.23097426169838,0,7427
DeleteEvent_count,94149,0.8161318760687846,9.985161303878002,0,1069
DeploymentEvent_count,94149,0.0,0.0,0,0
DeploymentStatusEvent_count,94149,0.0,0.0,0,0


In [24]:
'''numeric_data = churn_data.select(numeric_features).toPandas()
axs = pd.scatter_matrix(numeric_data, figsize=(8, 8));
n = len(numeric_data.columns)
for i in range(n):
    v = axs[i, 0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = axs[n-1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())'''

"numeric_data = churn_data.select(numeric_features).toPandas()\naxs = pd.scatter_matrix(numeric_data, figsize=(8, 8));\nn = len(numeric_data.columns)\nfor i in range(n):\n    v = axs[i, 0]\n    v.yaxis.label.set_rotation(0)\n    v.yaxis.label.set_ha('right')\n    v.set_yticks(())\n    h = axs[n-1, i]\n    h.xaxis.label.set_rotation(90)\n    h.set_xticks(())"

In [45]:
churn_data = churn_data.withColumn("first_event", to_timestamp(churn_data.first_event))
churn_data = churn_data.withColumn("time_between_first_last_event", 
                                   datediff(churn_data.last_event, churn_data.first_event))
churn_data = churn_data.withColumn("last_event", to_timestamp(churn_data.last_event))
churn_data = churn_data.withColumn("created_at", to_timestamp(churn_data.created_at))

In [46]:
pd.DataFrame(churn_data.take(5), columns=churn_data.columns)

,login,_c0,followers_count,following_count,bio,blog,company,created_at,public_repos_count,public_gists_count,...,ReleaseEvent_count,RepositoryEvent_count,RepositoryImportEvent_count,RepositoryVulnerabilityAlertEvent_count,SecurityAdvisoryEvent_count,StatusEvent_count,TeamEvent_count,TeamAddEvent_count,WatchEvent_count,second_period_event_count
0,0loky0,0loky0,0,1,None,None,None,2011-05-24 20:15:25+00:00,7,0,...,0,0,0,0,0,0,0,0,0,19
1,1xPdd,1xPdd,0,0,None,None,None,2014-07-11 03:45:56+00:00,0,0,...,0,0,0,0,0,0,0,0,1,3
2,3nawa-kun,3nawa-kun,0,0,None,None,None,2016-01-28 02:35:21+00:00,1,0,...,0,0,0,0,0,0,0,0,0,365
3,Abberbex,Abberbex,1,0,None,None,None,2011-12-06 22:19:40+00:00,2,0,...,0,0,0,0,0,0,0,0,0,5
4,AbdulRafay,AbdulRafay,0,2,what you seek is seeking you ~Rumi,https://stackoverflow.com/users/413670/rafay,None,2011-02-12 20:59:57+00:00,5,0,...,0,0,0,0,0,0,0,0,0,6


In [47]:
churn_data.head(2)

[Row(login='0loky0', _c0='0loky0', followers_count='0', following_count='1', bio=None, blog=None, company=None, created_at='2011-05-24 20:15:25+00:00', public_repos_count='7', public_gists_count='0', hireable='True', updated_at='2019-01-09 15:03:59+00:00', time_between_first_last_event=None, actor='0loky0', event_count=19, last_event=None, first_event=None, CheckRunEvent_count=0, CheckSuiteEvent_count=0, CommitCommentEvent_count=0, ContentReferenceEvent_count=0, CreateEvent_count=5, DeleteEvent_count=0, DeploymentEvent_count=0, DeploymentStatusEvent_count=0, DownloadEvent_count=0, FollowEvent_count=0, ForkEvent_count=2, ForkApplyEvent_count=0, asGitHubAppAuthorizationEvent_count=0, GistEvent_count=0, GollumEvent_count=0, InstallationEvent_count=0, InstallationRepositoriesEvent_count=0, IssueCommentEvent_count=0, IssuesEvent_count=0, LabelEvent_count=0, MarketplacePurchaseEvent_count=0, MemberEvent_count=0, MembershipEvent_count=0, MilestoneEvent_count=0, OrganizationEvent_count=0, OrgB

In [48]:
churn_data.printSchema()

root
 |-- login: string (nullable = true)
 |-- _c0: string (nullable = true)
 |-- followers_count: string (nullable = true)
 |-- following_count: string (nullable = true)
 |-- bio: string (nullable = true)
 |-- blog: string (nullable = true)
 |-- company: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- public_repos_count: string (nullable = true)
 |-- public_gists_count: string (nullable = true)
 |-- hireable: string (nullable = true)
 |-- updated_at: string (nullable = true)
 |-- time_between_first_last_event: integer (nullable = true)
 |-- actor: string (nullable = true)
 |-- event_count: integer (nullable = true)
 |-- last_event: timestamp (nullable = true)
 |-- first_event: timestamp (nullable = true)
 |-- CheckRunEvent_count: integer (nullable = true)
 |-- CheckSuiteEvent_count: integer (nullable = true)
 |-- CommitCommentEvent_count: integer (nullable = true)
 |-- ContentReferenceEvent_count: integer (nullable = true)
 |-- CreateEvent_count: integer (nulla

In [ ]:
df = df.select('age', 'job', 'marital', 'education', 'default', 'balance', 
               'housing', 'loan', 'contact', 'duration', 'campaign', 'pdays',
               'previous', 'poutcome', 'deposit')
cols = df.columns
df.printSchema()